In [1]:
# Install all required libraries

import sys
!{sys.executable} -m pip install pulp 

import pulp
# Import PuLP modeler functions
from pulp import * 

import numpy as np
import math as math
import networkx as nx

## The Decision Variables and Objective Function 

In [2]:
problem = LpProblem("CMST_Problem",LpMinimize)

In [3]:


# Graph of edges. The edges are the decision variables
E_weights = np.array([
 [0,7,9,14,0,15,5,1,0], #root
 [0,0,16,0,15,0,0,0,0], 
 [0,16,0,0,0,3,0,0,0],
 [0,0,0,0,0,0,2,11,6],
 [0,15,0,0,0,0,0,10,4],
 [0,3,0,0,0,0,8,0,0],
 [0,0,0,2,0,8,0,0,0],
 [0,0,0,11,10,0,0,0,0],
 [0,0,0,6,4,0,0,0,0]
])

# Capacity constraint
K = 3

V = np.array([
 [0,1,1,1,0,1,1,1,0], #root
 [0,0,1,0,1,0,0,0,0], #a
 [0,1,0,0,0,1,0,0,0], #b
 [0,0,0,0,0,0,1,1,1], #c
 [0,1,0,0,0,0,0,1,1], #d
 [0,1,0,0,0,0,1,0,0], #e
 [0,0,0,1,0,1,0,0,0], #f
 [0,0,0,1,1,0,0,0,0], #g
 [0,0,0,1,1,0,0,0,0]  #h
])

In [4]:
# Given sets V, V+, d, and given adjacency matrix E and capacity constraint K

# For the second constraint, we let d be a list of integers of size |V+|, with the entries being the demands of each node
# d = [4,7,2,3,4,9,...]

# d = the node weights
d = []
for i in range(0, len(n)): d.append(1)


# For the first constraint, we want the sum of all elements in each column of E, except column 0, to be equal to 1
# i.e. the sum of all elements in a column j must equal 1 for all j in V+
#numOfCols = np.size(E,1)



# Max number of subtrees in the graph
rootEdges = E[0,]
numSubtrees = 0

for n in rootEdges:
    if n != 0:
        numSubtrees += 1

# Min. # of subtrees needed to satisfy the capacity constraint
minNumSubtrees = math.ceil(sum(d)/K)
print(minNumSubtrees)

# The second constraint would be the number of subtrees >= the minimium number of subtrees necessary for K
problem += numSubtrees >= minNumSubtrees


print(problem)


CMST_Problem:
MINIMIZE
7*Edge_0_1 + 9*Edge_0_2 + 14*Edge_0_3 + 15*Edge_0_5 + 5*Edge_0_6 + 1*Edge_0_7 + 16*Edge_1_2 + 15*Edge_1_5 + 3*Edge_2_6 + 2*Edge_3_6 + 11*Edge_3_7 + 6*Edge_3_8 + 10*Edge_4_7 + 4*Edge_4_8 + 8*Edge_5_6 + 0
VARIABLES
Edge_0_1 Continuous
Edge_0_2 Continuous
Edge_0_3 Continuous
Edge_0_5 Continuous
Edge_0_6 Continuous
Edge_0_7 Continuous
Edge_1_2 Continuous
Edge_1_5 Continuous
Edge_2_6 Continuous
Edge_3_6 Continuous
Edge_3_7 Continuous
Edge_3_8 Continuous
Edge_4_7 Continuous
Edge_4_8 Continuous
Edge_5_6 Continuous



## The Constraints